In [ ]:
### Lab 14 Homework By Aidan Hennessy


In [ ]:
### Exercise 1 Part 1


In [ ]:
### In the six chunks of code immediately below, I upload some of the necessary libraries, 
### uploaded the necessary file, filtered the number of rows to 20,000, dropped na values and 
### preprocessed the text.


In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder


In [ ]:
pd.options.mode.chained_assignment = None


In [ ]:
ah_lab14_file = pd.read_csv('Reviews.csv')


In [ ]:
ah_lab14_file_red = ah_lab14_file[:20000]


In [ ]:
ah_lab14_file_red = ah_lab14_file_red.dropna()


In [ ]:
Lab14_stop_words = set(stopwords.words('english'))

def preprocess_text(text):
       text = re.sub(r'http\S+', '', text)
       text = re.sub(r'@\S+', '', text)
       text = re.sub(r'[^a-zA-Z\s]', '', text)
       text = text.lower()
       text = ' '.join([word for word in 
                        text.split() if word not in Lab14_stop_words])
       return text


In [ ]:
### In the three chunks of code immediately below, I create a file with two new columns,
### one with clean texts and one with tokenized texts.


In [ ]:
ah_lab14_file_red_two = ah_lab14_file_red


In [ ]:
ah_lab14_file_red_two['cleaned_text'] = ah_lab14_file_red_two['Text'].apply(preprocess_text)


In [ ]:
ah_lab14_file_red_two['tokenized_column'] = ah_lab14_file_red_two['cleaned_text'].apply(word_tokenize)


In [ ]:
### I import two more necessary libraries, then I filter the list of vocabulary words
### to just the ones that appear more than once and in fewer than 80 percent of the reviews. 
### Then I create two dataframes, "cleaned_articles" and "tokenized_articles"
### that will be useful in the next parts of this lab. 


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


In [ ]:
ah_lab14_e2_file = ah_lab14_file_red_two


In [ ]:
from collections import Counter

def create_vocabulary(data, min_freq, max_prop):

    ah_lab14_P2_words = []
    for text in data:
        ah_lab14_P2_tokens = word_tokenize(text.lower())  
        ah_lab14_P2_words.extend(ah_lab14_P2_tokens)
    
    word_counts = Counter(ah_lab14_P2_words)

    filtered_vocab = {}
    for word, count in word_counts.items():
        proportion = count / len(data) 
        if count >= min_freq and proportion <= max_prop:
            filtered_vocab[word] = count

    return filtered_vocab

vocab_data = ah_lab14_e2_file['cleaned_text']


vocabulary = create_vocabulary(vocab_data, 2, 0.8)


In [ ]:
cleaned_articles = ah_lab14_file_red_two['cleaned_text']


In [ ]:
tokenized_articles = ah_lab14_file_red_two['tokenized_column']


In [ ]:
### There is no Exercise 1 Part 2. 


In [ ]:
### Exercise 1 Part 3


In [ ]:
### Here I vectorize the data then fit transform it twice. Then, I transform the data into tfidf data.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


In [ ]:
ah_lab14_vec = CountVectorizer(analyzer='word')
ah_lab14_tdm = ah_lab14_vec.fit_transform(cleaned_articles)


In [ ]:
doc_term_matrix = ah_lab14_vec.fit_transform([" ".join(text) for text in tokenized_articles])


In [ ]:
ah_lab14_tfidf_transform = TfidfTransformer()
ah_lab14_tfidf_mat = ah_lab14_tfidf_transform.fit_transform(doc_term_matrix)


In [ ]:
### I import the NMF library and then I create five components aka five topics for 
### my first Non-Negative Matrix Factorization (NMF) model. I then fit transform the
### data before using the  "get_feature_names_out()." Then, I identify the 10 most 
### frequently appearing words in each of the five discovered NMF topics. 


In [ ]:
from sklearn.decomposition import NMF


In [ ]:
num_of_topics_five = 5
ah_lab14_nmf_five = NMF(n_components=num_of_topics_five)
ah_lab14_tfidf_mat_nmf_five = ah_lab14_nmf_five.fit_transform(ah_lab14_tfidf_mat)


In [ ]:
ah_lab14_terms = ah_lab14_vec.get_feature_names_out()


In [ ]:
for i, topic in enumerate(ah_lab14_nmf_five.components_):
    top_words_per_topic_idx = topic.argsort()[:-11:-1]
    top_words_per_topic = [ah_lab14_terms[idx] for idx in top_words_per_topic_idx]
    print(f"NMF Topic {i + 1}: {', '.join(top_words_per_topic)}")


In [ ]:
NMF Topic 1: great, product, good, love, like, chocolate, price, chips, taste, buy
NMF Topic 2: coffee, cup, strong, blend, bold, flavor, roast, keurig, kcups, like
NMF Topic 3: tea, green, teas, grey, loose, earl, iced, bags, love, flavor
NMF Topic 4: dog, treats, food, dogs, treat, loves, br, eat, one, cat
NMF Topic 5: juice, br, drink, sugar, soda, like, orange, taste, switch, sweet


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_score
import numpy as np
from sklearn.decomposition import LatentDirichletAllocation
from gensim.corpora.dictionary import Dictionary
from gensim.models.coherencemodel import CoherenceModel


In [ ]:
ah_lab14_lda_five = LatentDirichletAllocation(n_components=num_of_topics_five)
ah_lab14_tfidf_mat_lda_five = ah_lab14_lda_five.fit_transform(ah_lab14_tfidf_mat)


In [ ]:
for i, topic in enumerate(ah_lab14_lda_five.components_):
    top_words_per_topic_idx = topic.argsort()[:-11:-1]
    top_words_per_topic = [ah_lab14_terms[idx] for idx in top_words_per_topic_idx]
    print(f"LDA Topic {i + 1}: {', '.join(top_words_per_topic)}")


In [ ]:
LDA Topic 1: clam, chowder, cavenders, span, minsbr, classtiny, jumbo, cotton, length, ibs
LDA Topic 2: juice, chips, drink, soda, like, taste, flavor, peanut, br, butter
LDA Topic 3: lobster, pocky, cheetos, jerk, rainbow, ploy, camper, dulce, leche, bisque
LDA Topic 4: great, product, good, like, br, food, one, love, would, taste
LDA Topic 5: coffee, tea, cup, flavor, like, taste, good, strong, one, blend


In [ ]:
### In Part 3 and Part 4, I notice topics that are largely focused on food and drink. 
### The difference between the two is that the NMF model has one topic about cats and dogs
### while the LDA model has zero topics about cats and dogs. 


In [ ]:
### Exericsse 1 Part 5


In [ ]:
### I repeat everything I did in Part 3 and Part 4 in the four chunks of code immediately below
### for Part 5, but instead of their being five topics for the second LDA model and the second 
### NMF model, there are nine topics for the second LDA model and the second NMF model.


In [ ]:
num_of_topics_nine = 9
ah_lab14_nmf_nine = NMF(n_components=num_of_topics_nine)
ah_lab14_tfidf_mat_nine = ah_lab14_nmf_nine.fit_transform(ah_lab14_tfidf_mat)


In [ ]:
for i, topic in enumerate(ah_lab14_nmf_nine.components_):
    top_words_per_topic_idx = topic.argsort()[:-11:-1]
    top_words_per_topic = [ah_lab14_terms[idx] for idx in top_words_per_topic_idx]
    print(f"NMF Topic {i + 1}: {', '.join(top_words_per_topic)}")


In [ ]:
NMF Topic 1: like, br, good, one, taste, really, dont, tried, would, ive
NMF Topic 2: coffee, cup, strong, bold, blend, flavor, roast, smooth, bitter, keurig
NMF Topic 3: tea, green, teas, grey, loose, earl, iced, bags, flavor, love
NMF Topic 4: dog, treats, food, dogs, treat, loves, love, eat, cat, newmans
NMF Topic 5: juice, drink, soda, sugar, orange, switch, fruit, br, carbonated, sweet
NMF Topic 6: great, product, price, amazon, find, love, store, buy, order, local
NMF Topic 7: chips, salt, potato, flavor, bag, kettle, love, vinegar, chip, bags
NMF Topic 8: peanut, butter, pb, fat, calories, love, tastes, regular, protein, great
NMF Topic 9: chocolate, hot, cocoa, milk, dark, best, cup, love, flavor, keurig


In [ ]:
ah_lab14_lda_nine = LatentDirichletAllocation(n_components=num_of_topics_nine)
ah_lab14_tfidf_mat_lda_nine = ah_lab14_lda_nine.fit_transform(ah_lab14_tfidf_mat)


In [ ]:
for i, topic in enumerate(ah_lab14_lda_nine.components_):
    top_words_per_topic_idx = topic.argsort()[:-11:-1]
    top_words_per_topic = [ah_lab14_terms[idx] for idx in top_words_per_topic_idx]
    print(f"LDA Topic {i + 1}: {', '.join(top_words_per_topic)}")


In [ ]:
LDA Topic 1: sausages, bahlsen, earths, hairball, stong, brittle, triplets, bent, unappealing, tulleys
LDA Topic 2: olives, baronet, definatly, tourangelle, marzipan, reccommend, sprinkles, martinis, turkish, daves
LDA Topic 3: coffee, tea, flavor, like, taste, good, cup, drink, great, one
LDA Topic 4: mic, icicle, soil, gnats, usb, recording, potting, tast, rainbow, audio
LDA Topic 5: waffle, stonewall, waffles, loaf, costa, ants, belgian, span, classtiny, minsbr
LDA Topic 6: altoids, grinds, gunpowder, roastbr, sumatran, revolution, dulce, smashed, pear, leche
LDA Topic 7: popcorn, sprinkles, carrot, kernels, chips, virginia, jimmies, popper, skippy, oak
LDA Topic 8: great, like, good, product, br, food, love, one, taste, dog
LDA Topic 9: trays, beaba, merrick, dha, freezing, gelatin, coriander, earths, kracker, chest


In [ ]:
### In the LDA model and NMF model of Part 5, the NMF model's topics focus very strongly on food and drink. 
### But the LDA model's topics foucs a lot less on food and drink than the NMF model's topics do. 


In [ ]:
### Exercise 2 Part 1


In [ ]:
### Here I import the final libraries we need.


In [ ]:
import gensim
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from gensim.models import Nmf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_score
from sklearn.metrics import mean_squared_error


In [ ]:
### I upload a dictionary and a corpus to be used for our models. 


In [ ]:
dictionary = Dictionary(tokenized_articles)
corpus = [dictionary.doc2bow(text) for text in tokenized_articles]


In [ ]:
### This creates the Gensim version of an LDA model for when there are five topics
### and a Gensim version of an NMF model when there are five topics. 


In [ ]:
ah_lab14_Lda_five = LdaModel(corpus, num_topics=num_of_topics_five, id2word=dictionary)


In [ ]:
ah_lab14_Nmf_five_rev = Nmf(corpus, num_topics=num_of_topics_five, id2word=dictionary)


In [ ]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(cleaned_articles)


In [ ]:
### The two chunks of code immediately below calculate the first Sklearn LDA's logarithmic perplexity score
### and the first Sklearn NMF's RMSE score. They are not directly comparable but they are comparable
### enough that it is better to use an LDA model with a negative logarithmic perplexity score than 
### an NMF with an rmse of 0.005. While it would have been better to have used the NMF model if the 
### LDA model's perplexity score was greater than the NMF rmse of 0.005. But since this is not the case, the
### results here favors the LDA model. 


In [ ]:
log_perplexity_lda = ah_lab14_Lda_five.log_perplexity(corpus)

print(f"First LDA Logarithmic Perplexity Score: {log_perplexity_lda}")


In [ ]:
First LDA Logarithmic Perplexity Score: -8.055354816573612


In [ ]:
W_five = ah_lab14_nmf_five.transform(tfidf_matrix) 
H_five = ah_lab14_nmf_five.components_ 

mod_reconstructed_five = np.dot(W_five, H_five)

rmse_five = np.sqrt(mean_squared_error(tfidf_matrix.toarray(), mod_reconstructed_five))

print(f"First RMSE: {rmse_five}")


In [ ]:
First RMSE: 0.0052836857706265354


In [ ]:
### In the five chunks of code immediately below, I add the coherence model library.
### Then I calculate the tbe first Gensim LDA's CV Coherence score and UCI Coherence Score 
### as well as the first Gensim NMF's CV Coherence score and UCI Coherence socre. 


In [ ]:
from gensim.models.coherencemodel import CoherenceModel


In [ ]:
coherence_model_lda_cv = CoherenceModel(model=ah_lab14_Lda_five, texts=tokenized_articles, dictionary=dictionary, coherence='c_v')
coherence_score_lda_cv = coherence_model_lda_cv.get_coherence()

print(f"LDA CV Coherence Score: {coherence_score_lda_cv}")


In [ ]:
LDA CV Coherence Score: 0.3799830888124176


In [ ]:
coherence_model_nmf_cv = CoherenceModel(model=ah_lab14_Nmf_five_rev, texts=tokenized_articles, dictionary=dictionary, coherence='c_v')
coherence_score_nmf_cv = coherence_model_nmf_cv.get_coherence()

print(f"NMF CV Coherence Score: {coherence_score_nmf_cv}")


In [ ]:
NMF CV Coherence Score: 0.41088751301450044


In [ ]:
coherence_model_lda_uci = CoherenceModel(model=ah_lab14_Lda_five, texts=tokenized_articles, dictionary=dictionary, coherence='c_uci')
coherence_score_lda_uci = coherence_model_lda_uci.get_coherence()

print(f"LDA UCI Coherence Score: {coherence_score_lda_uci}")


In [ ]:
LDA UCI Coherence Score: -0.09891672119933814


In [ ]:
coherence_model_nmf_uci = CoherenceModel(model=ah_lab14_Nmf_five_rev, texts=tokenized_articles, dictionary=dictionary, coherence='c_uci')
coherence_score_nmf_uci = coherence_model_nmf_uci.get_coherence()

print(f"NMF UCI Coherence Score: {coherence_score_nmf_uci}")


In [ ]:
NMF UCI Coherence Score: -0.030902851346895827


In [ ]:
### In the four chunks of code immediately below, I calculate the tbe first Gensim LDA's 
### NPMI Coherence score and UMass Coherence Score as well as the first Gensim NMF's 
### NPMI Coherence score and UMass Coherence socre. 


In [ ]:
coherence_model_lda_npmi = CoherenceModel(model=ah_lab14_Lda_five, texts=tokenized_articles, dictionary=dictionary, coherence='c_npmi')
coherence_score_lda_npmi = coherence_model_lda_npmi.get_coherence()

print(f"LDA NPMI Coherence Score: {coherence_score_lda_npmi}")


In [ ]:
LDA NPMI Coherence Score: 0.0012342307914701741


In [ ]:
coherence_model_nmf_npmi = CoherenceModel(model=ah_lab14_Nmf_five_rev, texts=tokenized_articles, dictionary=dictionary, coherence='c_npmi')
coherence_score_nmf_npmi = coherence_model_nmf_npmi.get_coherence()

print(f"NMF NPMI Coherence Score: {coherence_score_nmf_npmi}")


In [ ]:
NMF NPMI Coherence Score: 0.007334133063895502


In [ ]:
coherence_model_lda_umass = CoherenceModel(model=ah_lab14_Lda_five, texts=tokenized_articles, dictionary=dictionary, coherence='u_mass')
coherence_score_lda_umass = coherence_model_lda_umass.get_coherence()

print(f"LDA UMass Coherence Score: {coherence_score_lda_umass}")


In [ ]:
LDA UMass Coherence Score: -2.011368268767135


In [ ]:
coherence_model_nmf_umass = CoherenceModel(model=ah_lab14_Nmf_five_rev, texts=tokenized_articles, dictionary=dictionary, coherence='u_mass')
coherence_score_nmf_umass = coherence_model_nmf_umass.get_coherence()

print(f"NMF UMass Coherence Score: {coherence_score_nmf_umass}")


In [ ]:
NMF UMass Coherence Score: -2.1324941058369515


In [ ]:
### The first Gensim NMF's CV Coherenece score is higher than the first Gensim's 
### LDA CV Coherence score. The higher score is better, so these results work 
### in the favor of the first Gensim NMF. 

### The first Gensim NMF's UCI Coherenece score is higher than the first Gensim's 
### LDA UCI Coherence score. The higher score is better, so these results work 
### in the favor of the first Gensim NMF.

### The first Gensim NMF's NPMI Coherenece score is higher than the first Gensim's 
### LDA NPMI Coherence score. The higher score is better, so these results work 
### in the favor of the first Gensim NMF. 

### The first Gensim LDA's Umass Coherenece score is higher than the first Gensim's 
### NMF UMass Coherence score. The lower score is better, so these results work 
### in the favor of the first Gensim NMF.


In [ ]:
### In the two chunks of code immediately below, I calculate the Silhouette score for the first Sklearn 
### LDA model and the second Sklearn NMF model. The result was that the first Sklearn LDA model has a 
### higher Silhouette score than the first Sklearn NMF model, a result favorable for the first Sklearn 
### LDA model. 


In [ ]:
silhouette_avg_lda = silhouette_score(ah_lab14_tfidf_mat_lda_five, np.argmax(ah_lab14_tfidf_mat_lda_five, axis=1)) 
print(f"First LDA Silhouette Score: {silhouette_avg_lda}")


In [ ]:
First LDA Silhouette Score: 0.7034540099531483


In [ ]:
silhouette_avg_nmf = silhouette_score(ah_lab14_tfidf_mat_nmf_five, np.argmax(ah_lab14_tfidf_mat_nmf_five, axis=1)) # Get the cluster assignment for each data point and use it to calculate Silhouette score
print(f"First NMF Silhouette Score: {silhouette_avg_nmf}")


In [ ]:
First NMF Silhouette Score: 0.45043858421447386


In [ ]:
### For the LDA and NMF models with five topics, the perplexity score-rmse comparison favors LDA,
### and that comes from sklearn. All four Coherence model scores favor NMF, and they are all in gensim. 
### And the Silhouette score comparison favors LDA when compared to NMF, and the Silhouette scores 
### both come from sklearn. Both sklearn results favor LDA but the gensim results favor NMF. There 
### are four gensim results and two sklearn results, so in this case, the NMF model is preferred. 


In [ ]:
### In the two chunks of code immediately below, we create the second Gensim LDA model 
### and the second Gensim NMF model. 


In [ ]:
ah_lab14_Lda_nine = LdaModel(corpus, num_topics=num_of_topics_nine, id2word=dictionary)


In [ ]:
ah_lab14_Nmf_nine_rev = Nmf(corpus, num_topics=num_of_topics_nine, id2word=dictionary)


In [ ]:
### This calculates the Sklearn LDA's logarithmic perplexity score and the Sklearn NMF's RMSE score. 
### The LDA model has a negative perplexity score while rmse is calculated to equal 0.005. While 
### these metrics are not entirely comparable, they are comparable enough that it is better to use
### an LDA model with a negative logarithmic perplexity score than an NMF model with an rmse of 0.005. 
### If the logarithmic perplexity score of the LDA model was greater than 0.005 and the NMF model's rmse stayed at 0.005,
### we would have gone with the NMF model. But the LDA model has a logarithmic property that is below 0.005, 
### so the LDA model wins here. 


In [ ]:
log_perplexity_lda_nine = ah_lab14_Lda_nine.log_perplexity(corpus)

print(f"Second LDA Logarithmic Perplexity Score: {log_perplexity_lda_nine}")


In [ ]:
Second LDA Logarithmic Perplexity Score: -8.442564232470644


In [ ]:
W_nine = ah_lab14_nmf_nine.transform(tfidf_matrix) 
H_nine = ah_lab14_nmf_nine.components_ 

mod_reconstructed_nine = np.dot(W_nine, H_nine)

rmse_nine = np.sqrt(mean_squared_error(tfidf_matrix.toarray(), mod_reconstructed_nine))

print(f"Second RMSE: {rmse_nine}")


In [ ]:
Second RMSE: 0.005270011251954697


In [ ]:
### In the four chunks of code immediately below, I calculate the tbe second Gensim LDA's 
### CV Coherence score and UCI Coherence Score as well as the second Gensim NMF's CV Coherence 
### score and UCI Coherence score.


In [ ]:
coherence_model_lda_cv_nine = CoherenceModel(model=ah_lab14_Lda_nine, texts=tokenized_articles, dictionary=dictionary, coherence='c_v')
coherence_score_lda_cv_nine = coherence_model_lda_cv_nine.get_coherence()

print(f"LDA CV Coherence Score Nine: {coherence_score_lda_cv_nine}")


In [ ]:
LDA CV Coherence Score Nine: 0.39641133795694006


In [ ]:
coherence_model_nmf_cv_nine = CoherenceModel(model=ah_lab14_Nmf_nine_rev, texts=tokenized_articles, dictionary=dictionary, coherence='c_v')
coherence_score_nmf_cv_nine = coherence_model_nmf_cv_nine.get_coherence()

print(f"NMF CV Coherence Score Nine: {coherence_score_nmf_cv_nine}")


In [ ]:
NMF CV Coherence Score Nine: 0.4640577058992772


In [ ]:
coherence_model_lda_uci_nine = CoherenceModel(model=ah_lab14_Lda_nine, texts=tokenized_articles, dictionary=dictionary, coherence='c_uci')
coherence_score_lda_uci_nine = coherence_model_lda_uci_nine.get_coherence()

print(f"LDA UCI Coherence Score Nine: {coherence_score_lda_uci_nine}")


In [ ]:
LDA UCI Coherence Score Nine: -1.321361157016826


In [ ]:
coherence_model_nmf_uci_nine = CoherenceModel(model=ah_lab14_Nmf_nine_rev, texts=tokenized_articles, dictionary=dictionary, coherence='c_uci')
coherence_score_nmf_uci_nine = coherence_model_nmf_uci_nine.get_coherence()

print(f"NMF UCI Coherence Score Nine: {coherence_score_nmf_uci_nine}")


In [ ]:
NMF UCI Coherence Score Nine: 0.04269615535667522


In [ ]:
### In the four chunks of code immediately below, I add the coherence model library.
### Then I calculate the tbe second Gensim LDA's NPMI Coherence score and UMass Coherence Score 
### as well as the second Gensim NMF's NPMI Coherence score and UMass Coherence score.


In [ ]:
coherence_model_lda_npmi_nine = CoherenceModel(model=ah_lab14_Lda_nine, texts=tokenized_articles, dictionary=dictionary, coherence='c_npmi')
coherence_score_lda_npmi_nine = coherence_model_lda_npmi_nine.get_coherence()

print(f"LDA NPMI Coherence Score Nine: {coherence_score_lda_npmi_nine}")


In [ ]:
LDA NPMI Coherence Score Nine: -0.034141667205064376


In [ ]:
coherence_model_nmf_npmi_nine = CoherenceModel(model=ah_lab14_Nmf_nine_rev, texts=tokenized_articles, dictionary=dictionary, coherence='c_npmi')
coherence_score_nmf_npmi_nine = coherence_model_nmf_npmi_nine.get_coherence()

print(f"NMF NPMI Coherence Score Nine: {coherence_score_nmf_npmi_nine}")


In [ ]:
NMF NPMI Coherence Score Nine: 0.022497966553568616


In [ ]:
coherence_model_lda_umass_nine = CoherenceModel(model=ah_lab14_Lda_nine, texts=tokenized_articles, dictionary=dictionary, coherence='u_mass')
coherence_score_lda_umass_nine = coherence_model_lda_umass_nine.get_coherence()

print(f"LDA UMass Coherence Score Nine: {coherence_score_lda_umass_nine}")


In [ ]:
LDA UMass Coherence Score Nine: -3.487895572670281


In [ ]:
coherence_model_nmf_umass_nine = CoherenceModel(model=ah_lab14_Nmf_nine_rev, texts=tokenized_articles, dictionary=dictionary, coherence='u_mass')
coherence_score_nmf_umass_nine = coherence_model_nmf_umass_nine.get_coherence()

print(f"NMF UMass Coherence Score Nine: {coherence_score_nmf_umass_nine}")


In [ ]:
NMF UMass Coherence Score Nine: -2.2915215454603572


In [ ]:
### The second Gensim NMF's CV Coherenece score is higher than the second Gensim's 
### LDA CV Coherence score. The higher score is better, so these results work 
### in the favor of the second Gensim NMF. 

### The second Gensim NMF's UCI Coherenece score is higher than the second Gensim's 
### LDA UCI Coherence score. The higher score is better, so these results work 
### in the favor of the second Gensim NMF.

### The second Gensim NMF's NPMI Coherenece score is higher than the second Gensim's 
### LDA NPMI Coherence score. The higher score is better, so these results work 
### in the favor of the first Gensim NMF. 

### The second Gensim LDA's Umass Coherenece score is higher than the second Gensim's 
### UMass Coherence score. The lower score is better, so these results work 
### in the favor of the first Gensim LDA.


In [ ]:
### In the two chunks of code immediately below, I calculate the Silhouette score for the second Sklearn 
### LDA model and the second Sklearn NMF model. The result was that the second Sklearn LDA model has a 
### higher Silhouette score than the second Sklearn NMF model, a result favorable for the second Sklearn 
### LDA model. 


In [ ]:
silhouette_avg_lda_nine = silhouette_score(ah_lab14_tfidf_mat_lda_nine, np.argmax(ah_lab14_tfidf_mat_lda_nine, axis=1)) 
print(f"Second LDA Silhouette Score: {silhouette_avg_lda_nine}")


In [ ]:
Second LDA Silhouette Score: 0.6676928757234112


In [ ]:
silhouette_avg_nmf_nine = silhouette_score(ah_lab14_tfidf_mat_nine, np.argmax(ah_lab14_tfidf_mat_nine, axis=1)) # Get the cluster assignment for each data point and use it to calculate Silhouette score
print(f"Second NMF Silhouette Score: {silhouette_avg_nmf_nine}")


In [ ]:
Second NMF Silhouette Score: 0.24972796948024104


In [ ]:
### In the LDA model and the NMF model where there are nine topics, the LDA's logarithmic perplexity score
### and the NMF's rmse that are both from sklearn have results that favor the LDA model.Three out of the four coherence
### scores from gensim favor the NMF model and one favors the LDA model. The Silhouette scores of these two models
### favor the LDA model. Both sklearn results favor LDA but three out of four gensim results favor NMF. There 
### are four gensim results and two sklearn results, so in this case, the LDA model and the NMF model tie in terms
### of preferred model. 


In [ ]:
### Please check out corresponding CSV file for more info on comparisons. 
